In [1]:
gcs_bucket="mickael-dm-stt"
gcs_transcripts_path="transcripts"
model="small"

In [2]:
fr_gcs_uris=!(gsutil ls gs://{gcs_bucket}/source/chirp/audio/fr-FR)
file_array=[]
for uri in fr_gcs_uris:
    file_array.append({"gcs_uri": uri,"language_code": "fr-FR"})

en_gcs_uris=!(gsutil ls gs://mickael-dm-stt/source/chirp/audio/en-US)
for uri in en_gcs_uris:
    file_array.append({"gcs_uri": uri,"language_code": "en-US"})
print(file_array)
number_of_files=len(file_array)
print(f"Number of files: {number_of_files}")

[{'gcs_uri': 'gs://mickael-dm-stt/source/chirp/audio/fr-FR/x8mmzj2.flac', 'language_code': 'fr-FR'}, {'gcs_uri': 'gs://mickael-dm-stt/source/chirp/audio/fr-FR/x8mv7he.flac', 'language_code': 'fr-FR'}, {'gcs_uri': 'gs://mickael-dm-stt/source/chirp/audio/fr-FR/x8n3e82.flac', 'language_code': 'fr-FR'}, {'gcs_uri': 'gs://mickael-dm-stt/source/chirp/audio/fr-FR/x8n3jhn.flac', 'language_code': 'fr-FR'}, {'gcs_uri': 'gs://mickael-dm-stt/source/chirp/audio/fr-FR/x8n3qv2.flac', 'language_code': 'fr-FR'}, {'gcs_uri': 'gs://mickael-dm-stt/source/chirp/audio/fr-FR/x8n3ssu.flac', 'language_code': 'fr-FR'}, {'gcs_uri': 'gs://mickael-dm-stt/source/chirp/audio/en-US/x58y7eg.flac', 'language_code': 'en-US'}, {'gcs_uri': 'gs://mickael-dm-stt/source/chirp/audio/en-US/x8n0jf1.flac', 'language_code': 'en-US'}, {'gcs_uri': 'gs://mickael-dm-stt/source/chirp/audio/en-US/x8n29uv.flac', 'language_code': 'en-US'}, {'gcs_uri': 'gs://mickael-dm-stt/source/chirp/audio/en-US/x8n38du.flac', 'language_code': 'en-US'}]

In [3]:
import re
from google.cloud import storage
import os

storage_client = storage.Client()
bucket = storage_client.bucket(gcs_bucket)

for file in file_array:
    regex=f"gs://{gcs_bucket}/(.*)/(\w+.flac)$"
    path,filename=re.search(regex,file["gcs_uri"]).group(1,2)
    local_uri=f"sound/{file['language_code']}/{filename}"
    # Fetch results from Cloud Storage
    blob = bucket.blob(path+"/"+filename)
    
    if not os.path.exists(f"sound/{file['language_code']}/"):
            os.makedirs(f"sound/{file['language_code']}/")
    blob.download_to_filename(local_uri)
    file["local_uri"]=local_uri
    
print(file_array)

[{'gcs_uri': 'gs://mickael-dm-stt/source/chirp/audio/fr-FR/x8mmzj2.flac', 'language_code': 'fr-FR', 'local_uri': 'sound/fr-FR/x8mmzj2.flac'}, {'gcs_uri': 'gs://mickael-dm-stt/source/chirp/audio/fr-FR/x8mv7he.flac', 'language_code': 'fr-FR', 'local_uri': 'sound/fr-FR/x8mv7he.flac'}, {'gcs_uri': 'gs://mickael-dm-stt/source/chirp/audio/fr-FR/x8n3e82.flac', 'language_code': 'fr-FR', 'local_uri': 'sound/fr-FR/x8n3e82.flac'}, {'gcs_uri': 'gs://mickael-dm-stt/source/chirp/audio/fr-FR/x8n3jhn.flac', 'language_code': 'fr-FR', 'local_uri': 'sound/fr-FR/x8n3jhn.flac'}, {'gcs_uri': 'gs://mickael-dm-stt/source/chirp/audio/fr-FR/x8n3qv2.flac', 'language_code': 'fr-FR', 'local_uri': 'sound/fr-FR/x8n3qv2.flac'}, {'gcs_uri': 'gs://mickael-dm-stt/source/chirp/audio/fr-FR/x8n3ssu.flac', 'language_code': 'fr-FR', 'local_uri': 'sound/fr-FR/x8n3ssu.flac'}, {'gcs_uri': 'gs://mickael-dm-stt/source/chirp/audio/en-US/x58y7eg.flac', 'language_code': 'en-US', 'local_uri': 'sound/en-US/x58y7eg.flac'}, {'gcs_uri': 

In [4]:
import whisper
import time
import re
from google.cloud import storage

whisper_model=whisper.load_model(model)

# Instantiates a Cloud Storage client
storage_client=storage.Client()
bucket = storage_client.bucket(gcs_bucket)

start_time = time.time()

for file in file_array:
    transcript=whisper_model.transcribe(file["local_uri"], fp16=False)
    print(f"Transcript for {file['gcs_uri']}: {transcript['text']}")

    transcript_path=f"{gcs_transcripts_path}/whisper_{model}"
    transcript_id=re.search("(\w+)\.flac$",file["gcs_uri"]).group(1)
    #print(f"gs://{gcs_bucket}/{transcript_path}/txt/{transcript_id}.txt")
    
    # Instantiates a Cloud Storage client
    blob=bucket.blob(f"{transcript_path}/txt/{transcript_id}.txt")
    blob.upload_from_string(transcript["text"])
    
duration = time.time() - start_time
print(f"Took {duration} to execute")

#result = whisper_model.transcribe("sound/fr-FR/x8n3qv2.flac", fp16=False)
#print(result["text"])

#gs://mickael-dm-stt/transcripts/chirp/txt/x8n3ssu.txt

/opt/conda/lib/python3.10/site-packages/whisper/timing.py:58: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def backtrace(trace: np.ndarray):
100%|███████████████████████████████████████| 461M/461M [00:10<00:00, 44.2MiB/s]


Transcript for gs://mickael-dm-stt/source/chirp/audio/fr-FR/x8mmzj2.flac:  On va d'abord parler du cas Eli Waïs si tu veux bien. Clairement, donc est du coup annoncé du côté de Strasbourg. On le rappelle, Strasbourg a été racheté par le consortium Blue Co qui a acheté il y a un peu plus d'un an maintenant Chelsea avec les conséquences qu'on connaît. Voilà alors, Waïs si on en parle c'est que ça paraît assez fou de voir Strasbourg se positionner dessus parce que à 20 ans il a montré de très belles choses en l'année dernière avec 19 buts. On sait qu'il y avait plusieurs gros noms dessus de jolies noms notamment Chelsea évidemment et que voilà ça va alors marchand d'estimer à 25 et 40 millions ce qui paraît il y a encore quelques temps des stommes complètement hallucinantes pour Strasbourg. Mais dans un monde normal c'est une somme à l'un des stommes pour Strasbourg mais le monde strachours jaune n'est plus normal. Exactement. Il y a déjà tu le record. Voilà exactement. Ils viennent d'ach